Dataset: DFEW (Dynamic Facial Expression in the Wild)

O DFEW é um dataset de larga escala para o reconhecimento de expressões faciais, com o diferencial de ser composto por vídeos em vez de imagens estáticas. Coletado a partir de clipes de filmes, reality shows e programas de TV, ele é considerado "in-the-wild" por apresentar cenários, iluminações e poses complexas e não controladas. O DFEW permite a análise da dinâmica temporal das emoções, capturando a evolução de uma expressão ao longo do tempo.

Principais Características
Composição: O dataset completo contém aproximadamente 16.000 clipes de vídeo. Ele já vem com uma divisão oficial entre conjuntos de treino e teste, o que garante a consistência e a comparabilidade dos resultados com outros trabalhos de pesquisa.

Qualidade e Contexto: A qualidade dos vídeos é geralmente boa (padrão de filme/TV), mas apresenta desafios realistas como mudanças bruscas de iluminação, oclusões parciais e movimento de câmara (motion blur). As expressões são frequentemente atuadas, mas inseridas em contextos de cena complexos.

Dados Técnicos: Os vídeos possuem durações variáveis, geralmente entre 0.5 e 4 segundos, com uma média de 100-120 frames por clipe. O dataset que será utilizado possui uma resolução padronizada de 224x224 pixels.

Classes de Emoção: O dataset é rotulado com as 7 emoções básicas universais, garantindo compatibilidade com a RAF-DB e o ExpW:
Raiva
Nojo
Medo
Felicidade
Tristeza
Surpresa
Neutro

Principal Desafio e Oportunidade: O principal desafio do DFEW é a necessidade de processar e analisar dados sequenciais, o que é computacionalmente mais intensivo. No entanto, a sua principal oportunidade é o fator de inovação, permitindo investigar se a informação temporal (a dinâmica do movimento facial) melhora a precisão do reconhecimento de emoções em comparação com a análise de imagens estáticas isoladas.

In [ ]:
import os
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

print("Bibliotecas para a análise do DFEW importadas com sucesso!")

In [ ]:
# --- Análise do Dataset DFEW ---

# ATENÇÃO: Verifique os caminhos e nomes exatos dos ficheiros na sua pasta 'data/raw/DFEW/'
# O nome da pasta dos vídeos e do ficheiro de rótulos pode variar após descompactar.
PATH_DFEW_BASE = './data/raw/DFEW'
# Vamos usar os vídeos já redimensionados que eles forneceram
PATH_DFEW_VIDEOS = os.path.join(PATH_DFEW_BASE, 'Clip', 'clip_224x224') 
# Caminho para a pasta com os rótulos descompactados
PATH_DFEW_LABELS_FOLDER = os.path.join(PATH_DFEW_BASE, 'EmoLabel_DataSplit')

# Nomes prováveis dos ficheiros de rótulos (verifique na sua pasta)
train_label_file = 'train_label.csv'
test_label_file = 'test_label.csv'

PATH_DFEW_TRAIN_LABELS = os.path.join(PATH_DFEW_LABELS_FOLDER, train_label_file)
PATH_DFEW_TEST_LABELS = os.path.join(PATH_DFEW_LABELS_FOLDER, test_label_file)

print(f"Caminho dos vídeos DFEW: {PATH_DFEW_VIDEOS}")
print(f"Caminho dos rótulos de treino: {PATH_DFEW_TRAIN_LABELS}")

try:
    df_dfew_train = pd.read_csv(PATH_DFEW_TRAIN_LABELS)
    df_dfew_test = pd.read_csv(PATH_DFEW_TEST_LABELS)
    
    # Adicionar uma coluna 'split' para fácil identificação
    df_dfew_train['split'] = 'train'
    df_dfew_test['split'] = 'test'

    # Juntar tudo num único dataframe para análise geral
    df_dfew = pd.concat([df_dfew_train, df_dfew_test], ignore_index=True)
    
    # Renomear colunas para um padrão (ajuste se o nome real no CSV for diferente)
    df_dfew.rename(columns={'Clip_Name': 'video_filename', 'Emo_Type': 'emotion_id'}, inplace=True, errors='ignore')
    
    # Mapear IDs de emoção para nomes (DFEW usa 1-7 como RAF-DB)
    emotion_mapping_dfew = {
        1: 'Raiva', 2: 'Nojo', 3: 'Medo', 4: 'Felicidade',
        5: 'Tristeza', 6: 'Surpresa', 7: 'Neutro'
    }
    df_dfew['emotion_name'] = df_dfew['emotion_id'].map(emotion_mapping_dfew)
    
    print("\nRótulos do DFEW (treino e teste) carregados com sucesso!")
    display(df_dfew.head())

except FileNotFoundError:
    print(f"ERRO: Ficheiros de rótulos não encontrados. Verifique os caminhos e os nomes dos ficheiros .csv após descompactar.")

In [ ]:
if 'df_dfew' in locals():
    # Visualizar a distribuição do conjunto de treino
    plt.figure(figsize=(12, 7))
    sns.countplot(x='emotion_name', 
                  data=df_dfew[df_dfew['split'] == 'train'], 
                  palette='rocket', 
                  hue='emotion_name', 
                  legend=False,
                  order = df_dfew[df_dfew['split'] == 'train']['emotion_name'].value_counts().index)
    plt.title('Distribuição de Emoções no Dataset DFEW (Treino)', fontsize=16)
    plt.xlabel('Emoção', fontsize=12)
    plt.ylabel('Quantidade de Vídeos', fontsize=12)
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
if 'df_dfew' in locals():
    # Escolher uma amostra aleatória do dataframe
    sample_video_info = df_dfew.sample(1).iloc[0]
    
    # O nome do ficheiro no CSV pode não ter a extensão, então vamos garantir que não a colocamos duas vezes
    video_filename = os.path.splitext(sample_video_info['video_filename'])[0] + ".mp4"
    emotion_name = sample_video_info['emotion_name']
    
    # Construir o caminho completo para o ficheiro de vídeo
    video_path = os.path.join(PATH_DFEW_VIDEOS, video_filename)
    
    print(f"--- Inspecionando Vídeo de Exemplo ---")
    print(f"Ficheiro: {video_filename}")
    print(f"Emoção Rotulada: {emotion_name}")
    print(f"Caminho completo: {video_path}")
    
    if os.path.exists(video_path):
        try:
            # Abrir o vídeo com OpenCV
            cap = cv2.VideoCapture(video_path)
            
            # Obter propriedades do vídeo
            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            fps = cap.get(cv2.CAP_PROP_FPS)
            
            print(f"\nResolução: {width}x{height} pixels")
            print(f"Total de Frames: {frame_count}")
            print(f"Taxa de Frames (FPS): {fps:.2f}")
            
            # Mostrar o primeiro frame como amostra
            ret, frame = cap.read()
            if ret:
                print("\nExibindo o primeiro frame do vídeo:")
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                plt.figure(figsize=(6, 5))
                plt.imshow(frame_rgb)
                plt.axis('off')
                plt.show()
            
            cap.release()
            
        except Exception as e:
            print(f"\nNão foi possível abrir ou processar o vídeo. Erro: {e}")
    else:
        print(f"\nERRO: O ficheiro de vídeo não foi encontrado no caminho especificado.")